# 105 LangGraph: Sequential Workflows - Multi-Node Graphs

**Workshop**: LangGraph 101 - Foundations  
**Duration**: ~35 minutes  
**Difficulty**: Intermediate  
**Prerequisites**: Notebooks 103 (Your First Graph), 104 (Complex State Management)

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Build multi-node graphs with sequential execution** - Create workflows with multiple processing steps that run in a specific order
2. **Chain nodes together using add_edge()** - Connect nodes to control execution flow explicitly
3. **Implement error handling across workflow steps** - Add robust exception handling to SCM workflows
4. **Understand state transformation through pipelines** - See how state evolves as it passes through multiple nodes
5. **Create complete SCM workflows** - Build production-ready Tag → Address → Group creation pipelines
6. **Visualize complex graph structures** - Use mermaid diagrams to understand multi-node workflows


## Prerequisites

Before starting this notebook, you should have:

- ✅ **Completed Notebook 103** - Understanding of basic LangGraph structure, state schemas, and single-node graphs
- ✅ **Completed Notebook 104** - Knowledge of complex state management with multiple fields and data types
- ✅ **Python 3.11+** - Environment with required packages installed
- ✅ **Basic Python skills** - Understanding of functions, dictionaries, and type hints


## Table of Contents

1. [What You've Learned So Far](#what-youve-learned-so-far)
2. [Sequential Multi-Node Graphs](#sequential-multi-node-graphs)
   - 2.1 [Define State for Sequential Workflow](#define-state-for-sequential-workflow)
   - 2.2 [Create First Node](#create-first-node)
   - 2.3 [Create Second Node - The Critical Pattern!](#create-second-node)
   - 2.4 [Build the Sequential Graph](#build-the-sequential-graph)
   - 2.5 [Visualize the Sequential Graph](#visualize-the-sequential-graph)
   - 2.6 [Run the Sequential Workflow](#run-the-sequential-workflow)
3. [Exercise: Three-Node SCM Address Object Workflow](#exercise)
4. [Complete Example: SCM Tag → Address → Group Workflow](#complete-example)
5. [Error Handling in SCM Workflows](#error-handling)
6. [Summary](#summary)
7. [What's Next](#whats-next)


<a id='what-youve-learned-so-far'></a>

## 1. What You've Learned So Far

Before we dive into sequential multi-node workflows, let's recap what you've mastered in the previous notebooks.

### From Notebook 103: Your First Graph - Foundation

You learned the fundamentals of LangGraph:

- **State Schemas** - How to define state structure using TypedDict to track workflow data
- **Node Functions** - Creating Python functions that process state and return updates
- **Basic Graphs** - Building simple `START → node → END` workflows
- **Graph Compilation** - Using `compile()` to create executable applications
- **Invocation** - Running graphs with `invoke()` and passing initial state
- **Visualization** - Using mermaid diagrams to see graph structure

**Key Pattern from 103**:
```python
# Define state
class MyState(TypedDict):
    field: str

# Create node
def process(state: MyState) -> dict:
    return {"field": "processed"}

# Build graph
graph = StateGraph(MyState)
graph.add_node("process", process)
graph.set_entry_point("process")
graph.set_finish_point("process")
app = graph.compile()
```

### From Notebook 104: Complex State Management

You learned how to work with sophisticated state structures:

- **Multi-field State Schemas** - Defining state with multiple fields tracking different aspects
- **Diverse Data Types** - Working with strings, lists, integers, booleans, and dictionaries
- **Safe State Access** - Understanding how to safely read and update state fields
- **Optional Fields** - Handling fields that may or may not be initialized
- **SCM Object Structures** - Applying complex state to real network automation scenarios

**Key Pattern from 104**:
```python
class ComplexState(TypedDict):
    name: str
    items: List[str]
    count: int
    result: str

def process_complex(state: ComplexState) -> dict:
    # Read multiple fields
    name = state["name"]
    items = state["items"]
    
    # Process and return updates
    return {
        "count": len(items),
        "result": f"{name} has {len(items)} items"
    }
```

### What's New in This Notebook?

Now we combine these foundations to build **multi-node sequential workflows**:

- ⭐ **Multiple nodes** working together in a specific order
- ⭐ **Chaining nodes** using `add_edge()` to control flow
- ⭐ **State transformation** through a pipeline of processing steps
- ⭐ **Real-world SCM workflows** that mirror production configuration pipelines
- ⭐ **Error handling** across multiple workflow steps

This is where LangGraph becomes truly powerful for network automation!

### Network Admin Analogy

Think of this progression like building firewall rules:

- **Notebook 103** - Creating a single basic rule (one step)
- **Notebook 104** - Adding complex criteria to that rule (richer data)
- **Notebook 105** - Building a complete policy with multiple dependent rules (multi-step workflow)

Just as firewall policies have rules that must be evaluated in order, LangGraph workflows have nodes that execute sequentially!

Let's get started building your first multi-node workflow!


## 1.1 Environment Setup

Let's import everything we need for this workshop:


In [ ]:
# Core LangGraph imports
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, List, Optional

# Visualization
from IPython.display import Image, display

# For simulating SCM API errors in examples
import random

print("✅ Imports successful!")
print("\nReady to build sequential multi-node workflows!")

---

<a id='sequential-multi-node-graphs'></a>


## 2. Sequential Multi-Node Graphs

Now we're getting to the real power of LangGraph - **connecting multiple nodes** to create sequential workflows!

### What We're Learning

Up until now, you've built graphs with a single node:
```
START → node → END
```

But what if you need **multiple processing steps** that run in a specific order? Like:
```
START → validate → create → verify → END
```

This is called a **sequential workflow** - nodes execute one after another, each building on the work of the previous node.

### What We're Building

In this section, we'll build a simple two-node SCM address object workflow:

1. **Node 1 (Validate)**: Check if address object name is valid
2. **Node 2 (Create)**: Create the address object in SCM

Then we'll expand to a complete three-node Tag → Address → Group workflow!

### Network Admin Analogy

Think of this like a **configuration pipeline**:

- **Step 1**: Validate syntax and format
- **Step 2**: Stage configuration changes
- **Step 3**: Commit to running config
- **Step 4**: Verify changes applied

Each step **must complete** before the next one runs. That's exactly what sequential workflows do!

Let's build one!


<a id='define-state-for-sequential-workflow'></a>

### 2.1 Define State for Sequential Workflow

First, let's define our state. We'll keep it simple with four string fields to track our SCM address object creation workflow.

In [ ]:
class AddressCreationState(TypedDict):
    """State for tracking SCM address object creation workflow."""
    name: str               # Address object name
    ip_netmask: str         # IP address with netmask
    folder: str             # SCM folder location
    workflow_log: str       # Cumulative workflow log

print("✅ AddressCreationState defined!")
print("\nState structure:")
print("  - name: str")
print("  - ip_netmask: str")
print("  - folder: str")
print("  - workflow_log: str")
print("\n💡 All string fields - we've already learned how to handle complex types!")

<a id='create-first-node'></a>

### 2.2 Create First Node

Let's create our first node that validates the address input:

**Key Pattern**: This node will start building the workflow log. Notice we're creating a NEW log, not reading an old one (because `workflow_log` will be empty string initially).

In [ ]:
def validate_address_input(state: AddressCreationState) -> dict:
    """Node 1: Validate address object configuration input.

    Args:
        state: Current workflow state

    Returns:
        dict: Partial state update with initial workflow log
    """
    # Read input data
    name = state["name"]
    ip_netmask = state["ip_netmask"]
    folder = state["folder"]

    # Start the workflow log (NEW report, not reading old one)
    log = f"Step 1: Validated input - Name: '{name}', IP: {ip_netmask}, Folder: {folder}. "

    return {"workflow_log": log}

print("✅ validate_address_input defined!")
print("\n💡 This is Node 1 - it CREATES the workflow_log")

<a id='create-second-node'></a>

### 2.3 Create Second Node - The Critical Pattern!

Now let's create our second node. **Pay close attention here** - this is where many people make a mistake!

⚠️ **Common Mistake**: In a sequential workflow, the second node needs to **PRESERVE** what the first node added to the log, not replace it!

In [ ]:
def create_in_scm(state: AddressCreationState) -> dict:
    """Node 2: Simulate creating address object in SCM.

    Args:
        state: Current workflow state

    Returns:
        dict: Partial state update appending to workflow log
    """
    # Read existing log from state
    existing_log = state["workflow_log"]
    name = state["name"]

    # Simulate SCM API call (in production: client.address.create())
    new_log_entry = f"Step 2: Created address object '{name}' in SCM successfully."

    # APPEND to existing log (don't replace it!)
    updated_log = existing_log + new_log_entry

    return {"workflow_log": updated_log}

print("✅ create_in_scm defined!")
print("\n💡 This is Node 2 - it READS existing log and APPENDS to it")

<a id='build-the-sequential-graph'></a>

### 2.4 Build the Sequential Graph

Now comes the new part: connecting nodes together using `add_edge()`!

**The Pattern**:
1. Create the graph
2. Add BOTH nodes
3. Set entry point (connects START to first node)
4. **NEW**: Use `add_edge()` to connect first node to second node
5. Set finish point (connects last node to END)
6. Compile

In [ ]:
# Step 1: Create the graph
address_creation_graph = StateGraph(AddressCreationState)

# Step 2: Add BOTH nodes
address_creation_graph.add_node("validate", validate_address_input)
address_creation_graph.add_node("create", create_in_scm)

# Step 3: Set entry point (START → validate)
address_creation_graph.set_entry_point("validate")

# Step 4: 🌟 CONNECT THE NODES using add_edge()
address_creation_graph.add_edge("validate", "create")

# Step 5: Set finish point (create → END)
address_creation_graph.set_finish_point("create")

# Step 6: Compile
address_creation_app = address_creation_graph.compile()

print("✅ Sequential address creation workflow built and compiled!")
print("\n💡 Flow: START → validate → create → END")

<a id='visualize-the-sequential-graph'></a>

### 2.5 Visualize the Sequential Graph

Let's see our multi-node workflow:

In [ ]:
# Visualize the sequential workflow
display(Image(address_creation_app.get_graph().draw_mermaid_png()))

print("💡 Notice the flow: START → validate → create → END")

<a id='run-the-sequential-workflow'></a>

### 2.6 Run the Sequential Workflow

Let's invoke our multi-node graph and watch the state flow through both nodes:

In [ ]:
# Run the address creation workflow
result = address_creation_app.invoke({
    "name": "internal_network",
    "ip_netmask": "10.0.0.0/24",
    "folder": "Texas",
    "workflow_log": ""
})

print("✅ Graph executed successfully!")
print("\nFinal state:")
print(f"Name: {result['name']}")
print(f"IP/Netmask: {result['ip_netmask']}")
print(f"Folder: {result['folder']}")
print(f"\nWorkflow Log:\n{result['workflow_log']}")

### 2.6.1 Understanding State Transformation

Let's visualize how state transforms as it flows through the sequential nodes:

**Initial State** (passed to graph):
```python
{
    "name": "internal_network",
    "ip_netmask": "10.0.0.0/24",
    "folder": "Texas",
    "workflow_log": ""  # Empty initially
}
```

**After Node 1 (validate)**:
```python
{
    "name": "internal_network",          # ← Unchanged (passed through)
    "ip_netmask": "10.0.0.0/24",         # ← Unchanged (passed through)
    "folder": "Texas",                   # ← Unchanged (passed through)
    "workflow_log": "Step 1: Validated input - Name: 'internal_network', IP: 10.0.0.0/24, Folder: Texas. "
                     # ↑ UPDATED by Node 1
}
```

**After Node 2 (create)** - Final State:
```python
{
    "name": "internal_network",          # ← Still unchanged
    "ip_netmask": "10.0.0.0/24",         # ← Still unchanged
    "folder": "Texas",                   # ← Still unchanged
    "workflow_log": "Step 1: Validated input - Name: 'internal_network', IP: 10.0.0.0/24, Folder: Texas. Step 2: Created address object 'internal_network' in SCM successfully."
                     # ↑ APPENDED by Node 2 (preserves Node 1's data!)
}
```

**Key Insight: State Accumulation**

Notice the **workflow_log** field:
1. **Node 1** creates the initial log entry
2. **Node 2** reads the existing log and APPENDS to it (doesn't replace!)

This is the **critical pattern** in sequential workflows:
- ✅ Input fields (`name`, `ip_netmask`, `folder`) pass through unchanged
- ✅ Output fields (`workflow_log`) accumulate data from each node
- ✅ Each node contributes its part to the final state

This pattern enables complex multi-step workflows where each node builds on previous results!

---

<a id='exercise'></a>

## 3. Exercise: Three-Node SCM Address Object Workflow

Time to practice building sequential workflows! This exercise builds on what you just learned.

**Challenge**: Create a three-node SCM address object creation workflow

**Requirements**:
- Create a state called `AddressWorkflowState` with FIVE fields:
  - `address_name` (str): Address object name
  - `ip_netmask` (str): IP address with netmask
  - `folder` (str): Target SCM folder
  - `created_by` (str): Administrator creating the object
  - `workflow_summary` (str): Cumulative summary built by each node

- Build THREE nodes in sequence:
  1. **Node 1** (`validate_input`): Validates the address input and starts the summary
     - Output: `"Address '{address_name}' validated. "`
  2. **Node 2** (`check_folder`): Validates the folder exists and appends to summary
     - Output: Append `"Folder '{folder}' verified. "`
  3. **Node 3** (`create_object`): Simulates SCM API creation and finalizes summary
     - Output: Append `"Created by {created_by} in SCM successfully."`

**Example Input/Output**:

```python
# Input:
{
    "address_name": "web_server",
    "ip_netmask": "192.168.1.10/32",
    "folder": "Texas",
    "created_by": "admin@example.com"
}

# Final Output workflow_summary:
"Address 'web_server' validated. Folder 'Texas' verified. Created by admin@example.com in SCM successfully."
```

**Hints**:
- You'll need to use `add_edge()` TWICE (to connect 3 nodes in sequence)
- Each node must READ the existing summary and APPEND to it (except Node 1 which creates it)
- Remember docstrings for each node function!
- Pattern: `graph.add_edge("node1", "node2")` then `graph.add_edge("node2", "node3")`

**Steps**:
1. Define `AddressWorkflowState` with 5 fields
2. Create `validate_input` node (starts summary)
3. Create `check_folder` node (appends to summary)
4. Create `create_object` node (appends to summary)
5. Build graph, add all 3 nodes
6. Set entry point to first node
7. Add edge from node1 to node2
8. Add edge from node2 to node3
9. Set finish point to last node
10. Compile
11. Visualize with IPython
12. Test with sample input

Try it yourself below!

In [ ]:
# Your code here!
# Build the three-node configuration workflow

# Step 1: Define ConfigWorkflowState

# Step 2: Create validate_admin node function

# Step 3: Create check_config_type node function

# Step 4: Create finalize_summary node function

# Step 5: Build graph and add all 3 nodes

# Step 6: Set entry point

# Step 7: Add edge from node1 to node2

# Step 8: Add edge from node2 to node3

# Step 9: Set finish point

# Step 10: Compile

# Step 11: Visualize

# Step 12: Test with sample input

### 3.1 Exercise Solution

Below is the complete solution for the three-node address workflow. Try the exercise yourself first before looking at this solution!

In [ ]:
# SOLUTION: Three-Node Address Workflow

# Step 1: Define AddressWorkflowState
class AddressWorkflowState(TypedDict):
    """State for tracking three-node address workflow."""
    address_name: str
    ip_netmask: str
    folder: str
    created_by: str
    workflow_summary: str

print("✅ Step 1: AddressWorkflowState defined with 5 fields")

# Step 2: Create validate_input node function
def validate_input(state: AddressWorkflowState) -> dict:
    """Node 1: Validate address input and start workflow summary.
    
    Args:
        state: Current workflow state
        
    Returns:
        dict: Partial state update with initial workflow summary
    """
    address_name = state["address_name"]
    
    # Start the workflow summary (NEW summary, not reading old one)
    summary = f"Address '{address_name}' validated. "
    
    return {"workflow_summary": summary}

print("✅ Step 2: validate_input node function created")

# Step 3: Create check_folder node function
def check_folder(state: AddressWorkflowState) -> dict:
    """Node 2: Verify folder exists and append to workflow summary.
    
    Args:
        state: Current workflow state
        
    Returns:
        dict: Partial state update appending to workflow summary
    """
    # Read existing summary from state
    existing_summary = state["workflow_summary"]
    folder = state["folder"]
    
    # Append new entry to existing summary
    new_entry = f"Folder '{folder}' verified. "
    updated_summary = existing_summary + new_entry
    
    return {"workflow_summary": updated_summary}

print("✅ Step 3: check_folder node function created")

# Step 4: Create create_object node function
def create_object(state: AddressWorkflowState) -> dict:
    """Node 3: Simulate SCM object creation and finalize workflow summary.
    
    Args:
        state: Current workflow state
        
    Returns:
        dict: Partial state update finalizing workflow summary
    """
    # Read existing summary from state
    existing_summary = state["workflow_summary"]
    created_by = state["created_by"]
    
    # Append final entry to existing summary
    new_entry = f"Created by {created_by} in SCM successfully."
    updated_summary = existing_summary + new_entry
    
    return {"workflow_summary": updated_summary}

print("✅ Step 4: create_object node function created")

# Step 5: Build graph and add all 3 nodes
address_workflow_graph = StateGraph(AddressWorkflowState)
address_workflow_graph.add_node("validate_input", validate_input)
address_workflow_graph.add_node("check_folder", check_folder)
address_workflow_graph.add_node("create_object", create_object)

print("✅ Step 5: Graph created and all 3 nodes added")

# Step 6: Set entry point
address_workflow_graph.set_entry_point("validate_input")

print("✅ Step 6: Entry point set to validate_input")

# Step 7: Add edge from node1 to node2
address_workflow_graph.add_edge("validate_input", "check_folder")

print("✅ Step 7: Edge added from validate_input to check_folder")

# Step 8: Add edge from node2 to node3
address_workflow_graph.add_edge("check_folder", "create_object")

print("✅ Step 8: Edge added from check_folder to create_object")

# Step 9: Set finish point
address_workflow_graph.set_finish_point("create_object")

print("✅ Step 9: Finish point set to create_object")

# Step 10: Compile
address_workflow_app = address_workflow_graph.compile()

print("✅ Step 10: Graph compiled successfully")

# Step 11: Visualize
print("\n💡 Step 11: Visualizing the three-node workflow...")
display(Image(address_workflow_app.get_graph().draw_mermaid_png()))

# Step 12: Test with sample input
print("\n💡 Step 12: Testing with sample input...")
result = address_workflow_app.invoke({
    "address_name": "web_server",
    "ip_netmask": "192.168.1.10/32",
    "folder": "Texas",
    "created_by": "admin@example.com",
    "workflow_summary": ""
})

print("\n" + "=" * 70)
print("EXERCISE SOLUTION OUTPUT")
print("=" * 70)
print(f"Address Name: {result['address_name']}")
print(f"IP/Netmask: {result['ip_netmask']}")
print(f"Folder: {result['folder']}")
print(f"Created By: {result['created_by']}")
print(f"\nWorkflow Summary:")
print(f"  {result['workflow_summary']}")
print("=" * 70)
print("\n✅ All 12 steps completed successfully!")

---

<a id='complete-example'></a>

## 4. Complete Example: SCM Tag → Address → Group Workflow

Let's put everything together with a **complete, realistic SCM configuration workflow** using proper 3-node separation.

This demonstrates the FULL pattern you'll use in production:
1. Separate nodes for each API call
2. Dependency validation
3. State accumulation
4. Simulated SCM API responses

### What We're Building

```
START → create_tag → create_address → create_group → END
```

Each node simulates an actual SCM API call from the `pan-scm-sdk`.

### 4.1 Define Complete SCM Configuration State

### 4.0 Production Pattern: ScmClient Initialization

Before building sequential workflows with real SCM API calls, you need to initialize the `ScmClient` once at the module level.

**Best Practice Pattern:**

In production LangGraph applications, initialize the SCM client once and reuse it across all nodes. This avoids creating multiple client instances and maintains connection efficiency.

Here's the production pattern you'll use in real workflows:

In [ ]:
# Production Pattern: Initialize ScmClient once at module level
# (This code is for demonstration - not executed in this notebook)

"""
from scm.client import Scm
import os

# Initialize once at the top of your module
scm_client = Scm(
    client_id=os.getenv("SCM_CLIENT_ID"),
    client_secret=os.getenv("SCM_CLIENT_SECRET"),
    tsg_id=os.getenv("SCM_TSG_ID")
)

# Then use in your node functions
def create_tag_production(state: CompleteSCMConfigState) -> dict:
    '''Production version using real ScmClient.'''
    
    # Use the module-level client (no need to create new instance)
    tag_response = scm_client.tag.create({
        "name": state["tag_name"],
        "color": state["tag_color"],
        "folder": state["folder"]
    })
    
    return {
        "tag_id": tag_response["id"],
        "tag_created": True,
        "workflow_log": f"Created tag '{state['tag_name']}' with ID: {tag_response['id']}\n"
    }

def create_address_production(state: CompleteSCMConfigState) -> dict:
    '''Production version using real ScmClient.'''
    
    # Reuse the same client instance
    address_response = scm_client.address.create({
        "name": state["address_name"],
        "ip_netmask": state["ip_netmask"],
        "folder": state["folder"],
        "tag": [state["tag_name"]]
    })
    
    existing_log = state["workflow_log"]
    return {
        "address_id": address_response["id"],
        "address_created": True,
        "workflow_log": existing_log + f"Created address '{state['address_name']}' with ID: {address_response['id']}\n"
    }
"""

print("💡 Production Pattern Explained:")
print("   1. Initialize ScmClient ONCE at module level")
print("   2. Store credentials in environment variables")
print("   3. Reuse the same client instance across all nodes")
print("   4. Each node makes API calls using the shared client")
print("\n📚 For learning purposes, we'll use simulated versions below")
print("   Real API integration will be covered in Notebooks 106-107")

In [ ]:
from typing import TypedDict, List, Optional

class CompleteSCMConfigState(TypedDict):
    """State for complete SCM tag → address → group workflow."""
    # Input configuration
    folder: str
    tag_name: str
    tag_color: str
    address_name: str
    ip_netmask: str
    group_name: str

    # Workflow status tracking
    tag_created: bool
    address_created: bool
    group_created: bool

    # API response IDs (simulated)
    tag_id: Optional[str]
    address_id: Optional[str]
    group_id: Optional[str]

    # Cumulative workflow log
    workflow_log: str

print("✅ CompleteSCMConfigState defined!")
print("\n💡 This state tracks the complete 3-step SCM configuration workflow")

### 4.2 Node 1: Create SCM Tag

**Simulates:** `client.tag.create({"name": "Production", "color": "Red", "folder": "Texas"})`

**References:** See [docs/examples/tags.py](../../docs/examples/tags.py) for real pan-scm-sdk tag creation patterns.

In [ ]:
def create_tag_node(state: CompleteSCMConfigState) -> dict:
    """Node 1: Create tag in SCM (simulates client.tag.create)."""
    import uuid

    tag_name = state["tag_name"]
    tag_color = state["tag_color"]
    folder = state["folder"]

    # Simulate API call
    tag_id = f"tag-{uuid.uuid4().hex[:8]}"

    # Create log entry
    log = f"[1/3] Created tag '{tag_name}' (color: {tag_color}) in folder '{folder}'\n      API Response: ID={tag_id}\n\n"

    return {
        "tag_created": True,
        "tag_id": tag_id,
        "workflow_log": log
    }

print("✅ create_tag_node defined")

### 4.3 Node 2: Create Address Object

**Simulates:** `client.address.create({"name": "web_server", "ip_netmask": "...", "tag": ["Production"], ...})`

**Key Pattern:** This node READS the existing log and APPENDS to it.

**References:** See [docs/examples/address_objects.py](../../docs/examples/address_objects.py) for real pan-scm-sdk address object creation patterns.

In [ ]:
def create_address_node(state: CompleteSCMConfigState) -> dict:
    """Node 2: Create address object in SCM (simulates client.address.create)."""
    import uuid

    # Dependency check
    if not state["tag_created"]:
        raise ValueError("❌ Cannot create address: tag must be created first!")

    address_name = state["address_name"]
    ip_netmask = state["ip_netmask"]
    folder = state["folder"]
    tag_name = state["tag_name"]
    existing_log = state["workflow_log"]

    # Simulate API call
    address_id = f"addr-{uuid.uuid4().hex[:8]}"

    # Create log entry
    new_entry = f"[2/3] Created address '{address_name}' ({ip_netmask}) in folder '{folder}'\n      Tagged with: [{tag_name}]\n      API Response: ID={address_id}\n\n"

    # APPEND to existing log
    updated_log = existing_log + new_entry

    return {
        "address_created": True,
        "address_id": address_id,
        "workflow_log": updated_log
    }

print("✅ create_address_node defined")

### 4.4 Node 3: Create Address Group

**Simulates:** `client.address_group.create({"name": "web_servers", "static": ["web_server"], ...})`

**Key Pattern:** Validates BOTH dependencies (tag AND address) before proceeding.

**References:** See [docs/examples/address_groups.py](../../docs/examples/address_groups.py) for real pan-scm-sdk address group creation patterns.

In [ ]:
def create_group_node(state: CompleteSCMConfigState) -> dict:
    """Node 3: Create address group in SCM (simulates client.address_group.create)."""
    import uuid

    # Dependency checks
    if not state["tag_created"]:
        raise ValueError("❌ Cannot create group: tag must be created first!")
    if not state["address_created"]:
        raise ValueError("❌ Cannot create group: address object must be created first!")

    group_name = state["group_name"]
    address_name = state["address_name"]
    folder = state["folder"]
    tag_name = state["tag_name"]
    existing_log = state["workflow_log"]

    # Simulate API call
    group_id = f"grp-{uuid.uuid4().hex[:8]}"

    # Create log entry
    new_entry = f"[3/3] Created address group '{group_name}' in folder '{folder}'\n      Members: [{address_name}]\n      Tagged with: [{tag_name}]\n      API Response: ID={group_id}\n\n✅ Complete SCM configuration workflow finished successfully!"

    # APPEND to existing log
    updated_log = existing_log + new_entry

    return {
        "group_created": True,
        "group_id": group_id,
        "workflow_log": updated_log
    }

print("✅ create_group_node defined")

### 4.5 Build the Complete SCM Configuration Graph

Now connect all three nodes in sequence with proper dependency flow:

In [ ]:
from langgraph.graph import StateGraph, START, END

# Create graph
complete_scm_graph = StateGraph(CompleteSCMConfigState)

# Add all three nodes
complete_scm_graph.add_node("create_tag", create_tag_node)
complete_scm_graph.add_node("create_address", create_address_node)
complete_scm_graph.add_node("create_group", create_group_node)

# Connect in sequence
complete_scm_graph.set_entry_point("create_tag")              # START → create_tag
complete_scm_graph.add_edge("create_tag", "create_address")   # create_tag → create_address
complete_scm_graph.add_edge("create_address", "create_group") # create_address → create_group
complete_scm_graph.set_finish_point("create_group")           # create_group → END

# Compile
complete_scm_app = complete_scm_graph.compile()

print("✅ Complete SCM configuration graph built and compiled!")
print("\n💡 Flow: START → create_tag → create_address → create_group → END")

### 4.6 Visualize the Workflow

In [ ]:
# Visualize the complete 3-node workflow
display(Image(complete_scm_app.get_graph().draw_mermaid_png()))

print("💡 This graph shows the sequential dependency chain for SCM configuration")

### 4.7 Execute the Complete Workflow

Let's run the entire workflow and watch the state flow through all three nodes!

In [ ]:
# Execute the complete SCM configuration workflow
result = complete_scm_app.invoke({
    # Input configuration
    "folder": "Texas",
    "tag_name": "Production",
    "tag_color": "Red",
    "address_name": "web_server",
    "ip_netmask": "192.168.1.10/32",
    "group_name": "web_servers",

    # Initial state values
    "tag_created": False,
    "address_created": False,
    "group_created": False,
    "tag_id": None,
    "address_id": None,
    "group_id": None,
    "workflow_log": ""
})

print("=" * 70)
print("COMPLETE SCM CONFIGURATION WORKFLOW")
print("=" * 70)
print(result["workflow_log"])
print("=" * 70)
print("\nFinal State Summary:")
print(f"  ✅ Tag Created: {result['tag_created']} (ID: {result['tag_id']})")
print(f"  ✅ Address Created: {result['address_created']} (ID: {result['address_id']})")
print(f"  ✅ Group Created: {result['group_created']} (ID: {result['group_id']})")
print("\n🎉 All three SCM objects created successfully in sequence!")

### 4.8 Key Insights: Production-Ready SCM Workflows

**What Makes This Production-Ready:**

1. ✅ **Proper Node Separation**
   - Each node handles ONE API call
   - Clean separation of concerns
   - Easy to test and debug

2. ✅ **Dependency Validation**
   - Nodes check prerequisites before executing
   - Prevents invalid configurations
   - Clear error messages

3. ✅ **State Accumulation**
   - Workflow log builds up across all nodes
   - Complete audit trail
   - Debugging visibility

4. ✅ **Realistic API Simulation**
   - Mirrors actual `pan-scm-sdk` patterns
   - Uses correct config structures
   - Returns simulated IDs like real API

**Real-World SCM SDK Equivalents:**

This workflow simulates:
```python
from scm.client import Scm

client = Scm(client_id="...", client_secret="...", tsg_id="...")

# Step 1: Create tag
tag = client.tag.create({
    "name": "Production",
    "color": "Red",
    "folder": "Texas"
})

# Step 2: Create address object
address = client.address.create({
    "name": "web_server",
    "ip_netmask": "192.168.1.10/32",
    "folder": "Texas",
    "tag": ["Production"]
})

# Step 3: Create address group
group = client.address_group.create({
    "name": "web_servers",
    "static": ["web_server"],
    "folder": "Texas",
    "tag": ["Production"]
})
```

**Dependency Tracking Demonstrated:**

Notice how each node validates its dependencies:
- **Node 2 (create_address)** checks `state["tag_created"]` before proceeding
- **Node 3 (create_group)** checks BOTH `state["tag_created"]` AND `state["address_created"]`

This ensures objects are created in the correct order and prevents invalid SCM configurations!

**Next Steps:**

You now understand sequential workflows with dependency tracking! In **Notebook 106**, we'll add **conditional routing** to handle different configuration scenarios (e.g., production vs development folders).

---

<a id='error-handling'></a>

## 5. Error Handling in SCM Workflows

In production SCM workflows, API calls can fail for various reasons. Let's learn how to handle errors gracefully in LangGraph nodes.

### Common SCM Exceptions

The `pan-scm-sdk` raises specific exceptions (from [docs/examples/address_objects.py](../../docs/examples/address_objects.py) lines 158-163):

- **`InvalidObjectError`** - Configuration data is invalid (e.g., bad IP format)
- **`NameNotUniqueError`** - Object name already exists in SCM
- **`ObjectNotPresentError`** - Referenced object doesn't exist
- **`MissingQueryParameterError`** - Required parameter missing

Let's build error-handling nodes that catch and handle these exceptions!

### 5.1 Define Error-Aware State

Our state needs fields to track errors and validation status:

In [ ]:
from typing import TypedDict, Optional, List

class ErrorAwareSCMState(TypedDict):
    """State for SCM workflow with error handling."""
    # Input
    address_name: str
    ip_netmask: str
    folder: str

    # Status tracking
    validation_passed: bool
    creation_succeeded: bool

    # Error tracking
    errors: List[str]              # List of error messages
    last_error_type: Optional[str] # Type of last error encountered

    # Results
    address_id: Optional[str]
    workflow_log: str

print("✅ ErrorAwareSCMState defined!")
print("\nNew fields for error handling:")
print("  - errors: List[str]           → Accumulates all error messages")
print("  - last_error_type: Optional[str] → Tracks error type for conditional routing")

### 5.2 Node 1: Validate with Error Handling

This node validates IP address format and catches **InvalidObjectError**:

In [ ]:
def validate_address_with_errors(state: ErrorAwareSCMState) -> dict:
    """Validate address object configuration with error handling.

    Simulates validation that would raise InvalidObjectError from SCM API.
    """
    import re

    address_name = state["address_name"]
    ip_netmask = state["ip_netmask"]
    folder = state["folder"]

    errors = []

    # Validate IP/netmask format
    ip_pattern = r'^(\d{1,3}\.){3}\d{1,3}/\d{1,2}$'
    if not re.match(ip_pattern, ip_netmask):
        # Simulate InvalidObjectError
        error_msg = f"❌ InvalidObjectError: IP/netmask '{ip_netmask}' has invalid format"
        errors.append(error_msg)

        return {
            "validation_passed": False,
            "errors": errors,
            "last_error_type": "InvalidObjectError",
            "workflow_log": f"Validation FAILED for '{address_name}'\n  Error: {error_msg}\n"
        }

    # Validate IP octets are 0-255
    ip_part = ip_netmask.split('/')[0]
    octets = [int(x) for x in ip_part.split('.')]
    if any(octet > 255 for octet in octets):
        error_msg = f"❌ InvalidObjectError: IP address octets must be 0-255"
        errors.append(error_msg)

        return {
            "validation_passed": False,
            "errors": errors,
            "last_error_type": "InvalidObjectError",
            "workflow_log": f"Validation FAILED for '{address_name}'\n  Error: {error_msg}\n"
        }

    # Validation passed
    log = f"✅ Validation PASSED for '{address_name}' ({ip_netmask}) in folder '{folder}'\n"

    return {
        "validation_passed": True,
        "errors": [],
        "last_error_type": None,
        "workflow_log": log
    }

print("✅ validate_address_with_errors defined!")
print("\n💡 This node catches InvalidObjectError conditions and returns error state")

### 5.3 Node 2: Create with Error Handling

This node attempts creation and handles **NameNotUniqueError** and **ObjectNotPresentError**:

In [ ]:
def create_address_with_errors(state: ErrorAwareSCMState) -> dict:
    """Create address object with error handling.

    Simulates SCM API call that could raise NameNotUniqueError or ObjectNotPresentError.
    Uses deterministic triggers based on input values for predictable testing.
    """
    import uuid

    address_name = state["address_name"]
    folder = state["folder"]
    existing_log = state["workflow_log"]

    # Check if validation passed first
    if not state["validation_passed"]:
        error_msg = "❌ Cannot create address: validation failed"
        return {
            "creation_succeeded": False,
            "errors": state["errors"] + [error_msg],
            "last_error_type": "PrerequisiteError",
            "workflow_log": existing_log + f"Creation SKIPPED: {error_msg}\n"
        }

    # Simulate NameNotUniqueError for addresses ending with "_duplicate"
    if address_name.endswith("_duplicate"):
        error_msg = f"❌ NameNotUniqueError: Address '{address_name}' already exists in SCM"
        return {
            "creation_succeeded": False,
            "errors": [error_msg],
            "last_error_type": "NameNotUniqueError",
            "workflow_log": existing_log + f"Creation FAILED\n  Error: {error_msg}\n"
        }

    # Simulate ObjectNotPresentError for folder "InvalidFolder"
    if folder == "InvalidFolder":
        error_msg = f"❌ ObjectNotPresentError: Folder '{folder}' does not exist in SCM"
        return {
            "creation_succeeded": False,
            "errors": [error_msg],
            "last_error_type": "ObjectNotPresentError",
            "workflow_log": existing_log + f"Creation FAILED\n  Error: {error_msg}\n"
        }

    # Success case
    address_id = f"addr-{uuid.uuid4().hex[:8]}"
    log = existing_log + f"✅ Creation SUCCEEDED\n  Created address '{address_name}' with ID: {address_id}\n"

    return {
        "creation_succeeded": True,
        "address_id": address_id,
        "errors": [],
        "last_error_type": None,
        "workflow_log": log
    }

print("✅ create_address_with_errors defined!")
print("\n💡 Deterministic error triggers:")
print("   - Names ending with '_duplicate' → NameNotUniqueError")
print("   - Folder 'InvalidFolder' → ObjectNotPresentError")
print("   - All other valid inputs → Success")

### 5.4 Build Error-Handling Workflow

Let's build a simple sequential workflow with error handling:

In [ ]:
from langgraph.graph import StateGraph, START, END

# Create graph
error_handling_graph = StateGraph(ErrorAwareSCMState)

# Add nodes
error_handling_graph.add_node("validate", validate_address_with_errors)
error_handling_graph.add_node("create", create_address_with_errors)

# Connect in sequence
error_handling_graph.set_entry_point("validate")
error_handling_graph.add_edge("validate", "create")
error_handling_graph.set_finish_point("create")

# Compile
error_handling_app = error_handling_graph.compile()

print("✅ Error-handling workflow built!")
print("\n💡 Flow: START → validate → create → END")
print("   (Errors are captured in state, workflow continues)")

### 5.5 Test 1: Valid Input (Success Path)

In [ ]:
# Test with valid input
result = error_handling_app.invoke({
    "address_name": "web_server_01",
    "ip_netmask": "192.168.1.10/32",
    "folder": "Texas",
    "validation_passed": False,
    "creation_succeeded": False,
    "errors": [],
    "last_error_type": None,
    "address_id": None,
    "workflow_log": ""
})

print("=" * 70)
print("TEST 1: VALID INPUT")
print("=" * 70)
print(result["workflow_log"])
print("=" * 70)
print(f"Validation Passed: {result['validation_passed']}")
print(f"Creation Succeeded: {result['creation_succeeded']}")
print(f"Errors: {result['errors']}")
if result['address_id']:
    print(f"Address ID: {result['address_id']}")

### 5.6 Test 2: Invalid IP Format (InvalidObjectError)

In [ ]:
# Test with invalid IP format
result = error_handling_app.invoke({
    "address_name": "web_server_02",
    "ip_netmask": "192.168.1.999/32",  # Invalid: octet > 255
    "folder": "Texas",
    "validation_passed": False,
    "creation_succeeded": False,
    "errors": [],
    "last_error_type": None,
    "address_id": None,
    "workflow_log": ""
})

print("=" * 70)
print("TEST 2: INVALID IP FORMAT")
print("=" * 70)
print(result["workflow_log"])
print("=" * 70)
print(f"Validation Passed: {result['validation_passed']}")
print(f"Creation Succeeded: {result['creation_succeeded']}")
print(f"Last Error Type: {result['last_error_type']}")
print(f"Errors: {result['errors']}")

### 5.7 Test 3: Malformed IP (InvalidObjectError)

In [ ]:
# Test with malformed IP
result = error_handling_app.invoke({
    "address_name": "web_server_03",
    "ip_netmask": "not-an-ip-address",  # Invalid format
    "folder": "Texas",
    "validation_passed": False,
    "creation_succeeded": False,
    "errors": [],
    "last_error_type": None,
    "address_id": None,
    "workflow_log": ""
})

print("=" * 70)
print("TEST 3: MALFORMED IP")
print("=" * 70)
print(result["workflow_log"])
print("=" * 70)
print(f"Validation Passed: {result['validation_passed']}")
print(f"Creation Succeeded: {result['creation_succeeded']}")
print(f"Last Error Type: {result['last_error_type']}")
print(f"Errors: {result['errors']}")

<a id='summary'></a>

---

## 6. Summary

Congratulations! You've mastered sequential multi-node workflows in LangGraph! 🎉

### What You Accomplished

1. **Built Multi-Node Graphs**
   - Created workflows with 2-3 nodes executing in sequence
   - Understood how state flows through multiple processing steps
   - Saw how each node builds on the work of previous nodes

2. **Chained Nodes with add_edge()**
   - Used `add_edge()` to explicitly control execution order
   - Connected START to first node, nodes to each other, and last node to END
   - Learned the difference between `add_edge()` and `set_entry_point()`/`set_finish_point()`

3. **Implemented Error Handling**
   - Added try/except blocks to handle SCM API exceptions
   - Tracked errors in state for debugging and reporting
   - Tested both success and failure paths

4. **Understood State Transformation**
   - Saw how state evolves as it passes through the pipeline
   - Each node reads state, processes data, and returns partial updates
   - Final state contains accumulated results from all nodes

5. **Created Complete SCM Workflows**
   - Built production-ready Tag → Address → Group workflows
   - Applied real-world dependency patterns (tags before addresses, addresses before groups)
   - Simulated actual `pan-scm-sdk` API usage patterns

6. **Visualized Complex Graphs**
   - Used mermaid diagrams to understand multi-node structure
   - Saw clear visual representation of workflow execution order

### Key Patterns You Learned

#### Sequential Workflow Pattern
```python
# Define state
class WorkflowState(TypedDict):
    input_data: str
    step1_result: str
    step2_result: str
    final_result: str

# Create nodes
def step1(state: WorkflowState) -> dict:
    result = f"Step 1 processed: {state['input_data']}"
    return {"step1_result": result}

def step2(state: WorkflowState) -> dict:
    # Builds on step1_result
    result = f"Step 2 processed: {state['step1_result']}"
    return {"step2_result": result}

# Build graph with add_edge()
graph = StateGraph(WorkflowState)
graph.add_node("step1", step1)
graph.add_node("step2", step2)
graph.add_edge(START, "step1")      # START → step1
graph.add_edge("step1", "step2")    # step1 → step2
graph.add_edge("step2", END)        # step2 → END
app = graph.compile()
```

#### Error Handling Pattern
```python
def create_with_error_handling(state: MyState) -> dict:
    """Node with robust error handling."""
    try:
        # Attempt operation
        result = create_scm_object(state["name"])
        return {
            "result": f"Success: {result}",
            "error": None
        }
    except Exception as e:
        return {
            "result": None,
            "error": f"Failed: {str(e)}"
        }
```

### Real-World Applications

The sequential workflow patterns you learned apply to many network automation scenarios:

- **Configuration Deployment**: Validate → Stage → Commit → Verify
- **Security Policy Creation**: Tag → Address → Service → Rule
- **Compliance Auditing**: Fetch Config → Parse Rules → Check Compliance → Generate Report
- **Batch Operations**: List Objects → Validate Each → Apply Changes → Log Results
- **Disaster Recovery**: Backup Config → Validate Backup → Test Restore → Store Archive

### Important Concepts

✅ **Sequential execution** - Nodes run in the order you define with `add_edge()`  
✅ **State accumulation** - Each node adds to state, building up results  
✅ **Error handling** - Use try/except in nodes to handle failures gracefully  
✅ **add_edge() vs set_entry_point()** - Both control flow, but add_edge is more explicit  
✅ **Dependency order** - Design workflows to match real-world dependencies (tags before addresses)  

### From Theory to Production

What we've built so far are **simulated workflows**. In production, you would:

1. **Replace simulations with real API calls**:
   ```python
   from scm.client import Scm
   
   def create_address_real(state: AddressState) -> dict:
       client = Scm(client_id="...", client_secret="...", tsg_id="...")
       response = client.address.create({
           "name": state["address_name"],
           "ip_netmask": state["ip_netmask"],
           "folder": state["folder"]
       })
       return {"address_id": response["id"]}
   ```

2. **Add comprehensive error handling** for different exception types
3. **Implement retry logic** for transient failures
4. **Add logging** for audit trails and debugging
5. **Validate inputs** before API calls
6. **Handle partial failures** in batch operations

You'll learn these production patterns in advanced notebooks!

### What Makes This Powerful

Sequential workflows enable:

- ✨ **Modular design** - Each node does one thing well
- ✨ **Reusability** - Nodes can be reused in different workflows
- ✨ **Testability** - Test each node independently
- ✨ **Visibility** - Visualize entire workflow structure
- ✨ **Maintainability** - Easy to modify or extend workflows
- ✨ **Error isolation** - Failures are contained and traceable

---

Great work! You now have the skills to build sophisticated sequential workflows. In the next notebook, you'll learn **conditional routing** - where workflows can take different paths based on runtime conditions!


<a id='whats-next'></a>

---

## 7. What's Next

Congratulations on mastering sequential multi-node workflows! You've built a strong foundation in LangGraph workflow design.

### What You've Mastered So Far

- **Notebook 103**: Basic graphs with single nodes
- **Notebook 104**: Complex state management with multiple fields
- **Notebook 105**: Sequential multi-node workflows with error handling ← You are here!

### Coming Up in Notebook 106: Conditional Routing - Dynamic Workflows

So far, your workflows follow a **fixed path** - every execution runs the same sequence of nodes:

```
START → validate → create → verify → END
```

But what if you need **different paths** based on runtime conditions?

```
START → validate → [SUCCESS or FAILURE?]
                    ↓                ↓
                  create          retry
                    ↓                ↓
                  verify          END
                    ↓
                  END
```

In **Notebook 106**, you'll learn:

1. **Conditional edges with add_conditional_edges()** - Routes that change based on state
2. **Router functions** - Logic that determines which path to take
3. **Dynamic workflows** - Different execution paths for different scenarios
4. **Advanced SCM patterns** - Smart retry logic, validation branches, approval workflows
5. **Error recovery** - Automatic retry and fallback strategies

### Real-World Conditional Workflow Example

Here's a preview of what you'll build:

```python
# Router function decides next step
def route_after_validation(state: State) -> str:
    if state["validation_passed"]:
        return "create_address"  # Success path
    else:
        return "log_error"       # Failure path

# Add conditional edge
graph.add_conditional_edges(
    "validate",                    # From node
    route_after_validation,        # Router function
    {
        "create_address": "create",  # If returns "create_address", go to "create" node
        "log_error": "error_handler" # If returns "log_error", go to "error_handler" node
    }
)
```

### Network Admin Analogy

Think of conditional routing like **ACL rule matching**:

- **Sequential workflows** (what you learned today): Rules evaluated top-to-bottom
- **Conditional routing** (next notebook): Different actions based on match criteria

Just as ACLs can permit/deny based on conditions, conditional workflows can route to different nodes based on state!

### Why This Matters for SCM Automation

Conditional routing enables intelligent workflows:

- **Retry logic**: Try again if API call fails
- **Approval workflows**: Route to human approval for sensitive changes
- **Environment-specific logic**: Different paths for dev/staging/prod
- **Validation branches**: Skip creation if validation fails
- **Error recovery**: Automatic fallback strategies

This is where your LangGraph workflows become truly intelligent and production-ready!

---

### Ready to Continue?

**Head to Notebook 106: Conditional Routing** to learn how to build dynamic, intelligent workflows that adapt to runtime conditions!

Great work completing this notebook! 🚀
